In [16]:
from sklearn.linear_model import LinearRegression
import numpy as np
from scipy import stats

In [17]:
%run "data_preperation.ipynb"

<>:1: SyntaxWarning: invalid escape sequence '\B'
C:\Users\arnet\AppData\Local\Temp\ipykernel_12248\4088575790.py:1: SyntaxWarning: invalid escape sequence '\B'
  file_path = 'data\BB_Rates_15min_from_10-2023.xlsx'


preperation complete!


C:\Users\arnet\AppData\Local\Temp\ipykernel_12248\2991408273.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  EURUSD_filtered['TimeFrame'] = EURUSD_filtered.groupby('Block').cumcount() + 1


In [25]:
def fit_model(block_data):
    X = np.arange(96).reshape(-1, 1)  # Independent variable (time steps)
    Y = block_data["Close"].values  # Dependent variable (closing prices)

    # Create and fit the model
    model = LinearRegression()
    model.fit(X, Y)
    return model, X, Y

In [19]:
def predict_CI(model, X, Y, confidence_level = 0.8):
    predictions = model.predict(X)
    # Predict the closing prices for t = 97 with confidence interval
    t = 97
    prediction = model.predict([[t]])[0]

    # Calculate the confidence interval
    n = len(X)
    t_value = stats.t.ppf((1 + confidence_level) / 2, n - 2)
    s = np.sqrt(np.sum((Y - predictions) ** 2) / (n - 2))
    s_x = np.sum((X - np.mean(X)) ** 2)
    s_y = s ** 2 * (1 + 1 / n + (t - np.mean(X)) ** 2 / s_x)
    confidence_interval = t_value * np.sqrt(s_y)
    return (prediction - confidence_interval), (prediction + confidence_interval)

In [ ]:
def revert_to_the_mean(closing_price, model, X, Y, mutex, bank):
    lower, upper = predict_CI(model, X, Y, confidence_level = 0.1) # inner CI
    lower_lower, upper_upper = predict_CI(model, X, Y, confidence_level = 0.8) # outer CI
    # we distinguish the two cases to take care of possible level shifts 
    if mutex == 0: # no investment made, potentially buy
        if closing_price["Close"] < lower and closing_price["Close"] > lower_lower:
            #trades += 1
            # print("Buy", closing_price["Close"] * bank, "$", closing_price["Dates"], closing_price["Close"])
            return 1, bank * closing_price["Close"]
        else:
            return 0, bank 
    else: # investment made, potentially sell, mutex == 1
        if closing_price["Close"] > upper and closing_price["Close"] < upper_upper:
            # print("Sell", (1/closing_price["Close"]) * bank, "€", closing_price["Dates"], closing_price["Close"])
            return 0, bank * (1/closing_price["Close"])
        else:
            return 1, bank
        